Chamando Bibliotecas

In [2]:
import pandas as pd
from sqlalchemy import create_engine

Declaração de Variáveis

In [3]:
# Declaração do caminho do arquivo com a base de dados
caminho_arquivo = "RAIS_VINC_PUB_NORTE.txt"
caminho_arquivo_csv = "arquivo_tratado.csv"

# Declaração das credenciais de acesso ao banco de dados do MySQL
host = "localhost"
usuario = "admin"
senha = "admin"
banco_de_dados = "dataviva_bd"
tabela = "rais_norte"
porta = "3306"

Tratamento da Base de Dados

In [ ]:
# Carrega o arquivo de dados
dados = pd.read_csv(caminho_arquivo, delimiter=";", encoding="ISO-8859-1")

# Elimina os registros com "Vínculo Ativo 31/12" igual a 0
dados = dados[dados["Vínculo Ativo 31/12"] != 0]

# Criar a nova variável "DIV CNAE 20" a partir da coluna "CNAE 2.0 Classe"
dados["DIV CNAE 20"] = dados["CNAE 2.0 Classe"].astype(str).str[:2]

# Salvando o arquivo tratado com a nova variável em csv
dados.to_csv(caminho_arquivo_csv, index=False)

Carregamento dos Dados em um Banco MySQL

In [ ]:
# Criar uma conexão com o MySQL
engine = create_engine(f"mysql://{usuario}:{senha}@{host}:{porta}/{banco_de_dados}")

# Lê o arquivo CSV
df = pd.read_csv(caminho_arquivo_csv)

# Carregar os dados no MySQL
df.to_sql(tabela, con=engine, if_exists="replace", index=False)

# Fechar a conexão
engine.dispose()


Análise dos Dados Salvos

In [6]:
# Criar uma conexão com o banco de dados MySQL
engine = create_engine(f"mysql://{usuario}:{senha}@{host}:{porta}/{banco_de_dados}")

# Faz uma query para descobrir as 5 atividades que mais empregam pessoas na Região Norte
query = """
SELECT `DIV CNAE 20`, COUNT(*) AS total_empregados
FROM {table_name}
GROUP BY `DIV CNAE 20`
ORDER BY total_empregados DESC
LIMIT 5
""".format(table_name=tabela)

df_atividades = pd.read_sql_query(query, engine)

# Faz uma query para descobrir os 5 municípios que mais empregam pessoas na Região Norte
query = """
SELECT `Município`, COUNT(*) AS total_empregados
FROM {table_name}
GROUP BY `Município`
ORDER BY total_empregados DESC
LIMIT 5
""".format(table_name=tabela)

df_municipios = pd.read_sql_query(query, engine)

# Fechar a conexão
engine.dispose()

# Exibir os resultados
print("As 5 atividades que mais empregam pessoas na Região Norte do Brasil:")
print(df_atividades)

print("Os 5 municípios que mais empregam pessoas na Região Norte do Brasil:")
print(df_municipios)

As 5 atividades que mais empregam pessoas na Região Norte do Brasil:
   DIV CNAE 20  total_empregados
0           84            954256
1           47            391892
2           86            100933
3           46             95302
4           85             93748
Os 5 municípios que mais empregam pessoas na Região Norte do Brasil:
   Município  total_empregados
0     130260            532312
1     150140            404718
2     172100            131645
3     110020            116923
4     120040            104800
